# Convergence tests of 2D Tree-Grid method

In [2]:
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import matplotlib.pyplot as plt
import TreeGrid_module as TG
import Numerics_module as num
import ExactBSPricing_module as ExBS

## Testing if the computational domain is large enough:

In [9]:
# creting small & large space-domain grids and time domain grid on the first refinement level:
X1=np.arange(0,144+8,8)
X1large=np.arange(0,400+8,8)
X2=np.arange(0,80+4,4)
X3=np.arange(30,50+2,2)
X=np.sort(np.unique(np.concatenate((X1,X2,X3))))
Xlarge=np.sort(np.unique(np.concatenate((X1large,X2,X3))))
Y=np.array(X)
Ylarge=np.array(Xlarge)
T=np.arange(0,0.25+0.01,0.01)
# number of refinement levels:
no_refinements=3 #max_no refinements: 5
# setting space step coefficient:
SpaceStepCoef=1/400

# running thorouh all refinement levels:
for k in range(no_refinements):
    # setting number of controls factor (actual number of controls: 8*factor-8):
    NCon=1+2*2**k
    # creating model & boundary objects for the problems and for the 1 dimensional problems that needs to be 
    # solved on the boundary:
    model_hjb=TG.SCP_uncertain_volatility_2D(no_controls_x=NCon, no_controls_y=NCon) 
    model_hjb_for_BC=TG.SCP_uncertain_volatility()
    model_bs=TG. SCP_BS_2D() 
    model_bs_for_BC=TG.SCP_BS()
    TCBC=TG.TCBC_butterfly_2D()
    TCBC_for_BC=TG.TCBC_butterfly()
    # solving HJB model (uncertain volatility model) on small domain:
    #   -computing solution on the left and on the lower boundary:
    solution_down=TG.TreeGridFull(X,T,model_hjb_for_BC,TCBC_for_BC)
    solution_left=TG.TreeGridFull(Y,T,model_hjb_for_BC,TCBC_for_BC)
    #   -computing solution of the 2D problem:
    BC_args=[solution_down,solution_left]
    Vhjb, Flags=TG.TreeGrid2D(X,Y,T,model_hjb,TCBC,BC_args,SpaceStepCoef)
    # solving BS model on small domain (in the same way):
    solution_down=TG.TreeGridFull(X,T,model_bs_for_BC,TCBC_for_BC)
    solution_left=TG.TreeGridFull(Y,T,model_bs_for_BC,TCBC_for_BC)
    BC_args=[solution_down,solution_left]
    Vbs, Flags=TG.TreeGrid2D(X,Y,T,model_bs,TCBC,BC_args,SpaceStepCoef)
    # solving HJB model (uncertain volatility model) on large domain (in the same way):
    solution_down=TG.TreeGridFull(Xlarge,T,model_hjb_for_BC,TCBC_for_BC)
    solution_left=TG.TreeGridFull(Ylarge,T,model_hjb_for_BC,TCBC_for_BC)
    BC_args=[solution_down,solution_left]
    Vhjb_large, Flags=TG.TreeGrid2D(Xlarge,Ylarge,T,model_hjb,TCBC,BC_args,SpaceStepCoef)
    # solving BS model on large domain (in the same way):
    solution_down=TG.TreeGridFull(Xlarge,T,model_bs_for_BC,TCBC_for_BC)
    solution_left=TG.TreeGridFull(Ylarge,T,model_bs_for_BC,TCBC_for_BC)
    BC_args=[solution_down,solution_left]
    Vbs_large, Flags=TG.TreeGrid2D(Xlarge,Ylarge,T,model_bs,TCBC,BC_args,SpaceStepCoef)
    # small and large domain meshgrids:
    Yy, Xx = np.meshgrid(Y, X)
    Yy_large, Xx_large = np.meshgrid(Ylarge, Xlarge)
    # priniting pairs of small/large domain solutions
    print( '%.12f' % Vhjb[(Xx==40)*(Yy==40)], '%.12f' % Vhjb_large[(Xx_large==40)*(Yy_large==40)],\
    '%.12f' % Vbs[(Xx==40)*(Yy==40)],'%.12f' % Vbs_large[(Xx_large==40)*(Yy_large==40)])
    
    # refinement of the grids:
    XX=(X[1:X.shape[0]]+X[0:X.shape[0]-1])/2
    X=np.sort(np.unique(np.concatenate((X,XX))))
    XXlarge=(Xlarge[1:Xlarge.shape[0]]+Xlarge[0:Xlarge.shape[0]-1])/2
    Xlarge=np.sort(np.unique(np.concatenate((Xlarge,XXlarge))))
    Y=np.array(X)
    Ylarge=np.array(Xlarge)
    TT=(T[1:T.shape[0]]+T[0:T.shape[0]-1])/2
    T=np.sort(np.unique(np.concatenate((T,TT))))

2.836402286431 2.836402286431 2.428345774590 2.428345774590
2.661865924012 2.661865924012 2.181764398614 2.181764398614
2.678366340837 2.678366340837 2.162118697040 2.162118697040


All pairs are equall in 8 decimal places, we can use Space domain [0,144]

## Convergence in 2D BS model

In [45]:
# list of all simulation that we will run:
simulations=list() 
simulations.append({'sigma_x':0.3, 'sigma_y':0.5, 'ro':0.4, 'SpaceStepCoef':1/400})
simulations.append({'sigma_x':0.05, 'sigma_y':0.05, 'ro':-0.95, 'SpaceStepCoef':1/400})
# number of refinement levels:
no_refinements=7 #max_no refinements: 7
# reserving space for lists of different X,Y,T grid refinements:
Xlist=[]
Ylist=[]
Tlist=[]
# creating grids for first refinement level:
X1=np.arange(0,144+8,8)
X2=np.arange(0,80+4,4)
X3=np.arange(30,50+2,2)
X=np.sort(np.unique(np.concatenate((X1,X2,X3))))
Y=np.array(X)
T=np.arange(0,0.25+0.01,0.01)
Xlist.append(X)
Ylist.append(Y)
Tlist.append(T)
# saving first space steps in X and Y. Reason: the exact solution computed with function from fExoticOptions R library
# can't be computed with zero asset price. Therefore, we will compute 1-norm error on the domain 
# [firststepX,max(X)] x [firststepY,max(Y)] 
firststepX=X[1]-X[0]
firststepY=Y[1]-Y[0]
# creting grids for all subsequent refinement levels:
for k in range(no_refinements-1):
    XX=(X[1:X.shape[0]]+X[0:X.shape[0]-1])/2
    X=np.sort(np.unique(np.concatenate((X,XX))))
    Y=np.array(X)
    TT=(T[1:T.shape[0]]+T[0:T.shape[0]-1])/2
    T=np.sort(np.unique(np.concatenate((T,TT))))
    Xlist.append(X)
    Ylist.append(Y)
    Tlist.append(T)
# reserving space for output; for each simulation we will have an array with number of X-space nodes (supposing number 
# of Y space nodes is the same) number of time nodes, value at [X,Y]=[40,40], Errors and EOCs for pointwise convergence
# and for L1 convergence
OUTPUT=np.zeros([len(simulations),no_refinements,7])
# simulation counter:
simnr=0
# running all simulations
for sim in simulations:
    # setting domain for computation of exact solution:
    Xex=X[X>=firststepX]
    Yex=Y[Y>=firststepY]
    # computing exact solution:
    Exsol=ExBS.ExactBSButterflyPrice(Xex,Yex,sigma_x=sim['sigma_x'], sigma_y=sim['sigma_y'], ro=sim['ro'])
    print("Exact solution for simulation nr. ",simnr," was computed.")
    # meshgrid for exact solution
    Yyex, Xxex = np.meshgrid(Yex, Xex)
    # setting space step coefficient, creting model & boundary objects for the problem and for the 
    # 1 dimensional problems that needs to be solved on the boundary
    SpaceStepCoef=sim['SpaceStepCoef']
    model_bs=TG.SCP_BS_2D(sigma_x=sim['sigma_x'], sigma_y=sim['sigma_y'], ro=sim['ro']) 
    TCBC=TG.TCBC_butterfly_2D()
    model_bs_for_BCX=TG.SCP_BS(sigma=sim['sigma_x'])
    model_bs_for_BCY=TG.SCP_BS(sigma=sim['sigma_y'])
    TCBC_for_BC=TG.TCBC_butterfly()
    # runing through all refinement levels:
    for k in range(no_refinements):
        # computing solution on the left and on the lower boundary
        solution_down=TG.TreeGridFull(Xlist[k],Tlist[k],model_bs_for_BCX,TCBC_for_BC)
        solution_left=TG.TreeGridFull(Ylist[k],Tlist[k],model_bs_for_BCY,TCBC_for_BC)
        # computing solution of the 2D problem
        BC_args=[solution_down,solution_left]
        Vbs, Flags=TG.TreeGrid2D(Xlist[k],Ylist[k],Tlist[k],model_bs,TCBC,BC_args,SpaceStepCoef)
        # meshgrid:
        Yy, Xx = np.meshgrid(Ylist[k], Xlist[k])
        # computing pointwise Error and EOC
        Err1_new=num.PointErrorExact2D(Xlist[k],Ylist[k],Vbs,ExactSol=Exsol[(Xxex==40)*(Yyex==40)][0],Point=[40,40])
        if k>0:
            EOC1=np.log((Err1/Err1_new))/np.log(2)
        else:
            EOC1=0
        Err1=Err1_new
        # computing L1 Error and EOC
        volumes=num.GridVolume2D(Xlist[k][Xlist[k]>=firststepX],Ylist[k][Ylist[k]>=firststepY])
        Err2_new=num.NormError(Norm=1,Sol=Vbs[2**k:,2**k:],RefSol=Exsol,Volumes=volumes,normalized=1)
        if k>0:
            EOC2=np.log((Err2/Err2_new))/np.log(2)
        else:
            EOC2=0
        Err2=Err2_new
        # printing results, and saving OUTPUT
        print(Vbs[(Xx==40)*(Yy==40)], Err1, EOC1, Err2, EOC2)
        OUTPUT[simnr,k,:]=[Tlist[k].shape[0]-1, Xlist[k].shape[0],Vbs[(Xx==40)*(Yy==40)][0], Err1, EOC1, Err2, EOC2]
        # saving output to .csv file
        filename="TG2D_Convergence_BS"+str(simnr)+".csv"
        np.savetxt(filename, OUTPUT[simnr,:,:])
    # incresing simulation counter:
    simnr=simnr+1
    print('\n ========================== \n')

Exact solution for simulation nr.  0  was computed.
[ 1.991] 0.176986729248 0 0.0120951908203 0
[ 1.821] 0.00701946663282 4.65613596557 0.00183574974747 2.7199922236
[ 1.823] 0.00887647519915 -0.338625490853 0.000716301822729 1.35772988321
[ 1.818] 0.00367413448594 1.2725824688 0.00030211210898 1.24548360319
[ 1.814] 5.30737951887e-05 6.11326090844 0.000124243655625 1.28191182607
[ 1.814] 0.000196441462866 -1.88802784974 5.10742406392e-05 1.28250442938
[ 1.814] 0.00013836701761 0.505599380828 2.5359167327e-05 1.01008847434


Exact solution for simulation nr.  1  was computed.
[ 3.362] 1.27887288092 0 0.0360134104995 0
[ 3.87] 0.770640922053 0.730742166292 0.019629311779 0.875524638875
[ 4.31] 0.331286488472 1.21797943278 0.00761864186745 1.36540384693
[ 4.634] 0.00690854760268 5.58355311212 0.000552706175308 3.7849492027
[ 4.647] 0.00573360270819 0.268940500857 3.85533282094e-05 3.8415854169
[ 4.647] 0.00597285354442 -0.0589784059725 4.60509858716e-05 -0.256376632583
[ 4.642] 0.0008346

## Convergence in 2D Uncertain volatility model

In [2]:
# list of all simulation that we will run:
simulations=list() 
simulations.append({'extrem':'max', 'SpaceStepCoef':1/400})
simulations.append({'extrem':'min', 'SpaceStepCoef':1/400})
# number of refinement levels:
no_refinements=4 #max_no refinements: 4
# reserving space for lists of different X,Y,T grid refinements:
Xlist=[]
Ylist=[]
Tlist=[]
# creting grids for first refinement level:
X1=np.arange(0,144+8,8)
X2=np.arange(0,80+4,4)
X3=np.arange(30,50+2,2)
X=np.sort(np.unique(np.concatenate((X1,X2,X3))))
Y=np.array(X)
T=np.arange(0,0.25+0.01,0.01)
Xlist.append(X)
Ylist.append(Y)
Tlist.append(T)
# creating grids for all subsequent refinement levels:
for k in range(no_refinements):
    XX=(X[1:X.shape[0]]+X[0:X.shape[0]-1])/2
    X=np.sort(np.unique(np.concatenate((X,XX))))
    Y=np.array(X)
    TT=(T[1:T.shape[0]]+T[0:T.shape[0]-1])/2
    T=np.sort(np.unique(np.concatenate((T,TT))))
    Xlist.append(X)
    Ylist.append(Y)
    Tlist.append(T)
# setting vector of numbers of controls factors for each refinement level (actual number of controls: 8*factor-8):    
NCon=1+2**np.arange(no_refinements+1)
# reserving space for output; for each simulation we will have an array with number of X-space nodes (supposing number 
# of Y space nodes is the same) number of time nodes, number of controls, value at [X,Y]=[40,40], Errors and EOCs 
# for pointwise convergence and for L1 convergence
OUTPUT=np.zeros([len(simulations),no_refinements,8])
# simulation counter:
simnr=0
# running all simulations
for sim in simulations:
    # setting space step coefficient, creating boundary object for the 1D problem and model and boundary objects 
    # for 1D problems that need to be solved on the boundary:
    SpaceStepCoef=sim['SpaceStepCoef']
    model_hjb_for_BCX=TG.SCP_uncertain_volatility(extrem=sim['extrem'])
    model_hjb_for_BCY=TG.SCP_uncertain_volatility(extrem=sim['extrem'])
    TCBC=TG.TCBC_butterfly_2D()
    TCBC_for_BC=TG.TCBC_butterfly()
    # computing reference solution:
    #   -creating model object with maximal number of controls
    model_hjb=TG.SCP_uncertain_volatility_2D(no_controls_x=NCon[-1], no_controls_y=NCon[-1],extrem=sim['extrem'])
    #   -computing solution on the left and on the lower boundary:
    solution_down=TG.TreeGridFull(Xlist[-1],Tlist[-1],model_hjb_for_BCX,TCBC_for_BC)
    solution_left=TG.TreeGridFull(Ylist[-1],Tlist[-1],model_hjb_for_BCY,TCBC_for_BC)
    #   -computing reference solution of the 2D problem:
    BC_args=[solution_down,solution_left]
    Refsol, Flags=TG.TreeGrid2D(Xlist[-1],Ylist[-1],Tlist[-1],model_hjb,TCBC,BC_args,SpaceStepCoef)
    print("Reference solution for uncertain volatility model was computed.")
    # runing through all refinement levels:
    for k in range(no_refinements):
        #   -creating model object with maximal number of controls
        model_hjb=TG.SCP_uncertain_volatility_2D(no_controls_x=NCon[k], no_controls_y=NCon[k], extrem=sim['extrem'])
        #   -computing solution on the left and on the lower boundary
        solution_down=TG.TreeGridFull(Xlist[k],Tlist[k],model_hjb_for_BCX,TCBC_for_BC)
        solution_left=TG.TreeGridFull(Ylist[k],Tlist[k],model_hjb_for_BCY,TCBC_for_BC)
        #   -computing solution of the 2D problem
        BC_args=[solution_down,solution_left]
        Vhjb, Flags=TG.TreeGrid2D(Xlist[k],Ylist[k],Tlist[k],model_hjb,TCBC,BC_args,SpaceStepCoef)
        # meshgrid:
        Yy, Xx = np.meshgrid(Ylist[k], Xlist[k])
        # computing pointwise Error and EOC
        Err1_new=num.PointError2D(Xlist[k],Ylist[k],Sol=Vhjb,RefSol=Refsol,Point=[40,40])
        if k>0:
            EOC1=np.log((Err1/Err1_new))/np.log(2)
        else:
            EOC1=0
        Err1=Err1_new
        # computing L1 Error and EOC
        volumes=num.GridVolume2D(Xlist[k],Ylist[k])
        Err2_new=num.NormError(Norm=1,Sol=Vhjb,RefSol=Refsol,Volumes=volumes,normalized=1)
        if k>0:
            EOC2=np.log((Err2/Err2_new))/np.log(2)
        else:
            EOC2=0
        Err2=Err2_new
        # printing results, and saving OUTPUT
        print(Vhjb[(Xx==40)*(Yy==40)], Err1, EOC1, Err2, EOC2)
        OUTPUT[simnr,k,:]=[Tlist[k].shape[0]-1, Xlist[k].shape[0], 8*NCon[k]-8, Vhjb[(Xx==40)*(Yy==40)][0],\
                     Err1, EOC1, Err2, EOC2]
        # saving output to .csv file
        filename="TG2D_Convergence_HJB"+str(simnr)+".csv"
        #np.savetxt(filename, OUTPUT[simnr,:,:])
    # incresing simulation counter:
    simnr=simnr+1
    print('\n ========================== \n')

Reference solution for uncertain volatility model was computed.
[ 2.75853815] 0.0812087027889028 0 0.0142834686353 0
[ 2.65677831] 0.02055114446630313 1.9824156052 0.00405072211594 1.81809534676
[ 2.67488639] 0.0024430611490338094 3.07245685858 0.00101327556665 1.99915254046


Reference solution for uncertain volatility model was computed.
[ 1.01152602] 0.09211758611011023 0 0.00493768863451 0
[ 0.95459468] 0.03518624837211326 1.38846490734 0.00220616758677 1.16229347819
[ 0.92897798] 0.009569544874891478 1.87848948202 0.000631495808782 1.80469732228




NameError: name 'model_hjb' is not defined